In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split


events_num_identifier = "_10_events"



path = "/nfs/dust/belle2/user/axelheim/MC_studies/my6modes/"
root_path = path + "rootfiles/wSim_wReco/"
#file_pi = uproot.open((root_path + "pi_nTuples_mode0.root"))

#file_raw = uproot.open(root_path + "mode0_17_events.root")

In [2]:
def conditions(s):
    label = -1
    if (int(s['genMotherPDG0']) == Bs_pdg) or (int(s['genMotherPDG1']) == Bs_pdg) or (int(s['genMotherPDG2']) == Bs_pdg) or (int(s['genMotherPDG3']) == Bs_pdg):
        label = 1
    elif (int(s['genMotherPDG0']) == Hc_pdg) or (int(s['genMotherPDG1']) == Hc_pdg) or (int(s['genMotherPDG2']) == Hc_pdg) or (int(s['genMotherPDG3']) == Hc_pdg):    
        label = 2
    else:
        label = 0
    
    return label

In [3]:
Bs_pdg_list = [-511, 511,-521,-521,521,521]
Hc_pdg_list = [-411, 411, -411, -421, 411, 423]
dfs =[]
for mode in range(6):
    print("mode:",mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode,mode)
    
    Bs_pdg = Bs_pdg_list[mode]
    Hc_pdg = Hc_pdg_list[mode]
    
    nm = f"_nTuples_mode{mode}" + events_num_identifier + ".root"
    raw_nTuples = uproot.concatenate([(root_path + "gamma" + nm), (root_path + "pi" + nm), (root_path + "K" + nm)])
    event = np.array(raw_nTuples["__event__"])
    mcPDG = np.array(raw_nTuples["mcPDG"])
    px = np.array(raw_nTuples["px"])
    py = np.array(raw_nTuples["py"])
    pz = np.array(raw_nTuples["pz"])
    E = np.array(raw_nTuples["E"])
    M = np.array(raw_nTuples["M"])
    charge = np.array(raw_nTuples["charge"])
    #mcPDG = np.array(raw_nTuples["kaonID"])
    #mcPDG = np.array(raw_nTuples["pionID"])
    genMotherPDG0 = np.array(raw_nTuples["genMotherPDG__bo0__bc"])
    genMotherPDG1 = np.array(raw_nTuples["genMotherPDG__bo1__bc"])
    genMotherPDG2 = np.array(raw_nTuples["genMotherPDG__bo2__bc"])
    genMotherPDG3 = np.array(raw_nTuples["genMotherPDG__bo3__bc"])
    
    #u,c = np.unique(event,return_counts=True)
    #print('\n entries per event:',u,c)
    
    
    df = pd.DataFrame({"event" : event,
                   "mcPDG" : mcPDG,
                   "px" : px,
                   "py" : py,
                   "pz" : pz,
                   "E" : E,
                   #"M" : M,
                   "charge" : charge,
                   "genMotherPDG0" : genMotherPDG0,
                   "genMotherPDG1" : genMotherPDG1,
                   "genMotherPDG2" : genMotherPDG2,
                   "genMotherPDG3" : genMotherPDG3})
    df['label'] = df.apply(conditions, axis=1)
    
    #print(df)
    
    dfs.append(df)
    print('\n')
    
    label_cut_names = ["BsHc","XHc","BsX","BsXHc"]
    for label_cut, label_cut_name in enumerate(label_cut_names):
        print(label_cut,label_cut_name)
        #print(df.describe())

        df_cut = df.copy()
        if label_cut != "BsXHc":
            df_cut = df_cut[df_cut.label != label_cut]
            
            #important to fix labels (no gap in label classes allowed for NN)
            if label_cut<2: 
                mask = df_cut.label == 2
                column_name = 'label'
                df_cut.loc[mask, column_name] = label_cut
                #df_cut[df_cut.label == 2] = label_cut

       # print(df_cut.describe())
        
        # read out data to save it
        event_tmp = df_cut.copy()        
        event_tmp = event_tmp[event_tmp.event == 1]
        #print(event_tmp.describe())
        
        num_FSPs_toData = len(event_tmp)
        numEvents = df_cut.event.max()
        num_features = 4 
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  

        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
        global_tag[:,-1] = str(mode)

        for i in range(numEvents):
            event_iter = i + 1
            #event_df = df_cut.copy()
            #event_df = event_df[event_df.event == event_iter]
            event_df = df_cut[df_cut.event == event_iter]

            for j in range(num_FSPs_toData):

                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]

                global_tag[i,j] = int(particle["mcPDG"])
                SA_target[i,j] = int(particle["label"])
        
        del event_tmp,df_cut

        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]

        data_subdir = "noSim_noReco_" + label_cut_name + events_num_identifier +  "/"        
        data_dir = Path("/nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/" + data_subdir)    
        data_dir.mkdir(parents=True, exist_ok=True)


        train_ratio = 0.75
        validation_ratio = 0.15
        test_ratio = 0.10

        x=leaves
        y=SA_target
        z=global_tag

        x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 


        np.save(data_dir / "leaves_train_{}.npy".format(mode), x_train)
        np.save(data_dir / "is_left_arr_train_{}.npy".format(mode), y_train)
        np.save(data_dir / "global_tag_train_{}.npy".format(mode), z_train)

        np.save(data_dir / "leaves_val_{}.npy".format(mode), x_val)
        np.save(data_dir / "is_left_arr_val_{}.npy".format(mode), y_val)
        np.save(data_dir / "global_tag_val_{}.npy".format(mode), z_val)

        np.save(data_dir / "leaves_test_{}.npy".format(mode), x_test)
        np.save(data_dir / "is_left_arr_test_{}.npy".format(mode), y_test)
        np.save(data_dir / "global_tag_test_{}.npy".format(mode), z_test)


        print("Data saved to:", data_dir,'\n')

mode: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


0 BsHc
numEvents: 15000
num_FSPs_toData: 11
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/noSim_noReco_BsHc_15000_events 

1 XHc
numEvents: 15000
num_FSPs_toData: 8
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/noSim_noReco_XHc_15000_events 

2 BsX
numEvents: 15000
num_FSPs_toData: 9
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/noSim_noReco_BsX_15000_events 

3 BsXHc
numEvents: 15000
num_FSPs_toData: 14
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/noSim_noReco_BsXHc_15000_events 

mode: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


0 BsHc
numEvents: 15000
num_FSPs_toData: 9
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/noSim_noReco_BsHc_15000_events 

1 XHc
numEvents: 15000
num_FSPs_toData: 4
Data saved to: /nfs/dust/belle2/user/axelheim/MC_studies/my6modes/data/noSim_noReco_XHc_15000_events 

2 BsX
numEvents: 150

In [4]:
print(df_cut.describe())


NameError: name 'df_cut' is not defined

In [ ]:
print(event_tmp)

In [ ]:
label_cut_names = ["BsHc","XHc","BsX","BsXHc"]
for i,df in enumerate(dfs):
    print("mode:",i,i,i,i,i,i,i,i,i,i,i,i,i,i,i,i)
    for label_cut, label_cut_name in enumerate(label_cut_names):
        print(label_cut,label_cut_name)

        if label_cut != "BsXHc":
            df_cut = df[df.label != label_cut]
        else:
            df_cut = df

        event_tmp = df_cut[df_cut.event == 1]    
        num_FSPs_toData = len(event_tmp)
        numEvents = df_cut.event.max()

        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)    

In [ ]:
df[(df.event==1)]

In [ ]:
df = pd.DataFrame({"event" : event,
               "mcPDG" : mcPDG,
               "px" : px,
               "py" : py,
               "pz" : pz,
               "E" : E,
               #"M" : M,
               "charge" : charge,
               "genMotherPDG0" : genMotherPDG0,
               "genMotherPDG1" : genMotherPDG1,
               "genMotherPDG2" : genMotherPDG2,
               "genMotherPDG3" : genMotherPDG3})
df['label'] = df.apply(conditions, axis=1)
df

In [ ]:

#df[(df.label==0) & (df['mode']==mode)]
subset = df[(df.event==1) & (df.label!=1)]
#df.loc[df['A'] > 2, 'B'] = new_val
#subset.__setitem__((subset.label == 2, 'label'), 1)
subset.__getitem__(subset.label == 2).__setitem__("label", 1)
#subset.loc[subset.label == 2,'label'] = 1
subset

In [ ]:
subset = df[(df.event==1) & (df.label!=1)]
subset.label == 2

In [ ]:
l = 14
print(leaves[l])
print(SA_target[l])
print(global_tag[l])

In [ ]:
event_df = df[df.event == 15]
event_df.iloc[3]["mcPDG"]

In [ ]:
event_df

In [ ]:
df = dfs[5]
event_df = df[df.event == 1]
event_df

In [ ]:
u,c = np.unique(event,return_counts=True)
print('\n entries per event:',u,c)

print(mcPDG)

print(event)